In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce
from pyspark.sql.types import StructType, StructField, StringType, LongType, StructType, IntegerType


# Initialize Spark session
spark = SparkSession.builder.appName("Interview").getOrCreate()


def get_df():
    # Define the schema for the DataFrame
    schema = StructType(
        [
            StructField("op", StringType(), True),
            StructField("ts", LongType(), True),
            StructField(
                "before",
                StructType(
                    [
                        StructField("id", IntegerType(), True),
                        StructField("name", StringType(), True),
                        StructField("age", IntegerType(), True),
                    ]
                ),
                True,
            ),
            StructField(
                "after",
                StructType(
                    [
                        StructField("id", IntegerType(), True),
                        StructField("name", StringType(), True),
                        StructField("age", IntegerType(), True),
                    ]
                ),
                True,
            ),
        ]
    )

    # Create some sample data
    data = [
        ("c", 178798797, None, {"id": 1, "name": "ayush", "age": 11}),
        (
            "u",
            178798798,
            {"id": 1, "name": "ayush", "age": 12},
            {"id": 1, "name": "ayush", "age": 13},
        ),
        ("c", 178798799, None, {"id": 2, "name": "john", "age": 25}),
        (
            "u",
            178798800,
            {"id": 2, "name": "john", "age": 25},
            {"id": 2, "name": "john", "age": 26},
        ),
    ]

    # Create DataFrame with the sample data
    return spark.createDataFrame(data, schema)


def get_cur_data(updated_data):
    # This function should return the current data DataFrame. For now, it's a placeholder.
    # Replace with actual logic to get current data.
    return spark.createDataFrame([], schema="id INT, name STRING, age INT")


def store(final_data):
    final_data.show()
    # Store the partition data using .write.partitionBy() if needed
    # Example: final_data.write.partitionBy("id").save("path/to/save")


# Data
data = get_df()

# Filter data for updates
updated_data = data.filter(col("op") == "u").orderBy(col("ts"))

# Filter data for creations
created_data = data.filter(col("op") == "c")

# Get current data and union with created data
curr_data = get_cur_data(updated_data).union(
    created_data.select(
        col("after.id").alias("id"),
        col("after.age").alias("age"),
        col("after.name").alias("name"),
    )
)

# Update the current data
final_data = updated_data.join(curr_data, col("after.id") == col("id"), "left").select(
    col("id"),
    coalesce(col("after.name"), col("name")).alias("name"),
    coalesce(col("after.age"), col("age")).alias("age"),
)

# Store the final data
store(final_data)
# spark.stop()

24/11/28 08:18:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/28 08:18:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|ayush| 13|
|  2| john| 26|
+---+-----+---+

